レース環境情報をスクレイピングで取得

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import datetime

pd.options.display.width = 500
pd.options.display.column_space = 200

In [ ]:
# race_id = "202104020101"
# url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
# html = requests.get(url)
# html.encoding= "EUC_JP"
# soup = BeautifulSoup(html.text, "html.parser")

In [ ]:
# 日付 = soup.find("dd",class_='Active')
# 日付.find("a").get_text()

In [ ]:
# レース情報 = soup.find("div", class_="RaceList_Item02") 

In [ ]:
# レース名 = レース情報.find("div",class_="RaceName").get_text()
# re.findall(r"\w+",レース名)

In [ ]:
# レース環境 = レース情報.find("div",class_="RaceData01").get_text()
# re.findall(r"\w+",レース環境)

----

スクレイピングしながらデータの成形

In [3]:
df = pd.read_pickle('df.pickle')
race_id_list = df.index.unique()
race_id_list

Int64Index([202103010101, 202103010102, 202103010103, 202103010104, 202103010105, 202103010106, 202103010107, 202103010108, 202103010109, 202103010110, 202103010111, 202103010112, 202101010101, 202101010102, 202101010103, 202101010104, 202101010105, 202101010106, 202101010107, 202101010108, 202101010109, 202101010110, 202101010111, 202101010112, 202102010101, 202102010102, 202102010103, 202102010104, 202102010105, 202102010106, 202102010107, 202102010108, 202102010109, 202102010110,
            202102010111, 202102010112],
           dtype='int64')

df.pickleに保存したdataframeのindexを参照してして全てのレース環境情報をスクレイピング

In [8]:
race_infos = {}
for race_id in tqdm(race_id_list):
  url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
  html = requests.get(url)
  html.encoding= "EUC_JP"
  soup = BeautifulSoup(html.text, "html.parser")

  日付 = soup.find("dd",class_='Active')
  日付 = 日付.find("a").get_text()
  レース情報 = soup.find("div", class_="RaceList_Item02") 
  # 距離 = レース情報.find("div",class_="RaceData01").find("span").get_text()
  # 馬場 = レース情報.find("div",class_="RaceData01").find("span",class_="Item03").get_text()
  レース環境 = レース情報.find("div",class_="RaceData01").get_text()
  info = re.findall(r"\w+",レース環境)
  info[0] = info[2][:1]
  info[1] = info[2][1:]
  info.pop(2)
  info_dict = {}
  # print(info)
  info_dict["date"] = f"{race_id[:4]}年{日付[:-3]}"

  for text in info:
    # print(text)
    if text in ["芝","ダ"]:
      # print("芝付き")
      info_dict["race_type"] = text
    if "障" in text:
      info_dict["race_type"] = "障害"
    if "m" in text:
      # print("mつき")
      info_dict["course_ren"] = re.findall(r"\d+",text)[0]
    if text in ["雨","小雨","小雪","雪","晴","曇"]:
      # print("天気")
      info_dict["weather"] = text
    # if "年" in text:
    #   info_dict["data"] = text
    if text in ["良","不良","重","稍","不"]:
      # print("馬場")
      info_dict["ground_state"] = text


  race_infos[race_id] = info_dict
  time.sleep(.5)

  0%|          | 0/36 [00:00<?, ?it/s]


TypeError: can only concatenate str (not "int") to str

In [1]:
race_infos_df = pd.DataFrame(race_infos).T
type(race_infos_df)


NameError: name 'pd' is not defined

In [ ]:
results = pd.read_pickle("df.pickle")

In [ ]:
results_addinfo = results.merge(race_infos_df,left_index=True,right_index=True,how="inner")

In [ ]:
results_addinfo.columns

In [ ]:
pd.set_option('display.max_rows',None)
# results_addinfo["ground_state"].value_counts()
results_addinfo.isnull().sum()

日付データの成形

In [ ]:
results_addinfo["date"] = pd.to_datetime(results_addinfo["date"],format="%Y年%m月%d日")
results_addinfo.info()

In [ ]:
results_addinfo.to_pickle("results_p")

In [ ]:
sorted_id_list = results_addinfo.sort_values("date").index.unique()

In [ ]:
round(len(sorted_id_list) * 0.7)

In [ ]:
# train_id_list = sorted_id_list[:round(len(sorted_id_list) * 0.7)]
# test_id_list = sorted_id_list[round(len(sorted_id_list) * 0.7):]

In [ ]:
# train = results_addinfo.loc[train_id_list]
# test = results_addinfo.loc[test_id_list]

In [ ]:
# len(train)

In [ ]:
# len(test)

In [ ]:
# len(results_addinfo)

In [ ]:
def split_data(df, test_size):
  # ソート
  sorted_id_list = df.sort_values('date').index.unique()
  # データの分割
  train_id_list = sorted_id_list[:round(len(sorted_id_list) * (1-test_size))]
  test_id_list = sorted_id_list[round(len(sorted_id_list) * (1-test_size)):]
  
  train = df.loc[train_id_list]
  test = df.loc[test_id_list]
  return train, test

In [ ]:
# ダミーデータの作成
results_d = pd.get_dummies(results_addinfo)
results_d["rank"] = results_d["着順"].map(lambda x: x if x < 4 else 4)
train,test = split_data(results_d,0.3)

In [ ]:
train.columns

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rank_1 = train["rank"].value_counts()[1]
rank_2 = train["rank"].value_counts()[2]
rank_3 = train["rank"].value_counts()[3]

rus = RandomUnderSampler(sampling_strategy="auto",random_state=None,replacement=False)

X_train = train.drop(["着順","date","rank"],axis=1)
y_train = train["rank"]
X_test = test.drop(["着順","date","rank"],axis=1)
y_test = test["rank"]


X_train_rus,y_train_rus = rus.fit_resample(X_train,y_train)

In [ ]:
X_train_rus

In [ ]:
y_train_rus

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train_rus,y_train_rus)

print(clf.score(X_train,y_train),clf.score(X_test,y_test))